In [13]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Gesell

This example demonstrates ...

Adapted from [unit 9: Gesell.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/Gesell.odc).



Note: not really working right now. Will go over it again later.



## Associated lecture video: Unit 9 Lesson 6

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=93" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

Gesell Developmental Schedules  measure  child development. The scoring is devised by the American child psychologist and pediatrician Arnold Gesell who founded the Clinic of Child Development at Yale in 1911 and directed it for many years.  The Gesell Developmental Schedules are a gauge of the status of a child's motor and language development and personal-social and adaptive behaviors.

For 21 children the age in months at which they first spoke and their Gesell Adaptive Score, which is the result of an aptitude test taken much later was recorded.  These data were originally collected by L. M. Linde of UCLA but were first published by M. R. Mickey, O. J. Dunn, and V. Clark, †Note on the use of stepwise regression in detecting outliers,† Computers and Biomedical Research, 1 (1967), pp. 105†111. 

The data have been used by several authors, e.g.,  N. R. Draper and
J. A. John, †Influential observations and outliers in regression,† 
Technometrics, 23 (1981), pp. 21†26.

Let X be the age in in months a child speaks his/her first word
and let Y be the Gesell adaptive score, a measure of a child†s
aptitude (observed later on). 


Correlation of r = ¢0.640 indicates a  moderately strong negative
relationship between age at first word spoken and Gesell score.
 How does the child†s aptitude change with how long it takes them to speak?

Are there any influential observations/potential outliers?

The solution follows a two stage modeling. 

(A) Obtain mean ICPOs. Outside BUGS take reciprocal of mean ICPOs.
(these are CPOs) and consder them as an input for model B.

(B) Read in and calibrate CPOs with posterior ordinates


In [20]:
# fmt: off
x = np.array(
    (15, 26, 10, 9, 15, 20, 18, 11, 8, 20, 7, 9, 10, 11, 11, 10, 12, 42, 17, 11, 10)
)

y = np.array(
    (95, 71, 83, 91, 102, 87, 93, 100, 104, 94, 113, 96, 83, 84, 102, 100, 105, 57, 121, 86, 100)
)
# fmt: on

## Model A

In [82]:
with pm.Model() as model:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.0001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "cpo", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    # what is the point of this when we want cpo anyways?
    # icpo = pm.Deterministic("icpo", 1 / f)
    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    trace = pm.sample(10000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 68 seconds.


In [83]:
cpo_df = az.summary(trace, var_names="cpo", round_to=5, filter_vars="like")

In [84]:
cpo_means = cpo_df["mean"].to_numpy()
cpo_means

array([0.03605, 0.02393, 0.01305, 0.02625, 0.02541, 0.03632, 0.03472,
       0.0355 , 0.03454, 0.0299 , 0.02114, 0.03432, 0.01305, 0.01677,
       0.03326, 0.03614, 0.02609, 0.02655, 0.00108, 0.02069, 0.03614])

## Model B


In [85]:
mlcpo = -2 * np.log(cpo_means)
MPQ = np.sum(mlcpo)

In [86]:
with pm.Model() as model:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.0001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "f", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    c = pm.Deterministic("c", cpo_means / f)
    # clean this up later
    g1 = pm.Deterministic("g1", c * pm.math.log(c))
    g2 = -pm.math.log(c)
    g3 = 0.5 * pm.math.abs_(c - 1)
    g4 = (c - 1) ** 2

    pm.Deterministic("s_g1", pm.math.sum(g1))
    pm.Deterministic("s_g2", pm.math.sum(g2))
    pm.Deterministic("s_g3", pm.math.sum(g3))
    pm.Deterministic("s_g4", pm.math.sum(g4))

    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    traceb = pm.sample(2000, target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 15 seconds.


In [88]:
az.summary(traceb, var_names="c", filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
c[0],1.033,0.180,0.735,1.374,0.003,0.002,3160.0,3569.0,1.0
c[1],1.167,0.619,0.556,2.134,0.009,0.006,4763.0,5757.0,1.0
c[2],1.188,0.684,0.453,2.246,0.010,0.007,4417.0,5430.0,1.0
c[3],1.052,0.272,0.657,1.562,0.004,0.003,4446.0,5257.0,1.0
c[4],1.048,0.231,0.698,1.486,0.003,0.002,7314.0,5868.0,1.0
c[5],1.034,0.186,0.733,1.390,0.003,0.003,3034.0,3420.0,1.0
c[6],1.035,0.189,0.731,1.393,0.003,0.002,3432.0,3875.0,1.0
c[7],1.035,0.185,0.706,1.360,0.003,0.002,3209.0,3483.0,1.0
c[8],1.041,0.204,0.701,1.399,0.004,0.003,3227.0,3260.0,1.0
c[9],1.049,0.243,0.684,1.498,0.004,0.003,4827.0,4504.0,1.0


In [89]:
az.waic(traceb, scale="deviance")

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1661: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 8000 posterior samples and 21 observations log-likelihood matrix.

              Estimate       SE
deviance_waic   165.60    10.14
p_waic            3.50        -

There has been a warning during the calculation. Please check the results.

In [1]:
%watermark --iversions -v

UsageError: Line magic function `%watermark` not found.
